### **Mở DRIVE**

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Cài đặt những thư viện cần thiết**

In [30]:
!pip install transformers datasets

In [31]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import os

### **Đọc Dataset**

In [32]:
import pandas as pd
from datasets import Dataset

# train_data_path = "/content/drive/MyDrive/dataset/Disaster Tweets/train.csv"
# test_data_path = "/content/drive/MyDrive/dataset/Disaster Tweets/test.csv"

train_data_path = '/content/train.csv'
test_data_path = '/content/test.csv'

# Load dataset
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

# Điền giá trị "unknown" cho NaN
df_train.fillna("unknown", inplace=True)
df_test.fillna("unknown", inplace=True)

df_train["combined_text"] = df_train["keyword"] + ";;" + df_train["location"] + ";;" + df_train["text"]
train_dataset = Dataset.from_pandas(df_train)

print(train_dataset["combined_text"])

['unknown;;unknown;;Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', 'unknown;;unknown;;Forest fire near La Ronge Sask. Canada', "unknown;;unknown;;All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected", 'unknown;;unknown;;13,000 people receive #wildfires evacuation orders in California ', 'unknown;;unknown;;Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ', 'unknown;;unknown;;#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires', 'unknown;;unknown;;#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas', "unknown;;unknown;;I'm on top of the hill and I can see a fire in the woods...", "unknown;;unknown;;There's an emergency evacuation happening now in the building across the street", "unknown;;unknown;;I'm afraid that the tornado is coming to our a

### **Load LLM và chỉnh lại trainer cho phù hợp**

In [33]:
model_name = "bert-base-multilingual-cased"  # Hoặc "bert-base-multilingual-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

### **Load Dataset Train**

In [34]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_function(examples):
    tokens = tokenizer(
        examples["combined_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens["labels"] = examples["target"]  # Giữ lại target
    return tokens

# Tokenize dataset
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["id", "keyword", "location", "text", "combined_text", "target"])


Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

### **Train time!!!**

In [ ]:
from transformers import TrainingArguments, Trainer
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./fine_tuned_sentiment",
    evaluation_strategy="no",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    remove_unused_columns=False  # 🔥 Cần để giữ input_ids, attention_mask
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()
trainer.save_model("./fine_tuned_sentiment")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-35-71c9a3164808>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.545400
1000,0.460400
1500,0.384800
2000,0.361500
2500,0.330400


### **In đáp án**

In [ ]:
import pandas as pd
from transformers import pipeline


label_id = {'LABEL_0' : 0, 'LABEL_1' : 1}
# Load dữ liệu test
df_test = pd.read_csv(test_data_path)

# Gộp keyword, location, text
df_test["combined_text"] = df_test["keyword"].fillna("") + " " + df_test["location"].fillna("") + " " + df_test["text"]

# Load mô hình gốc
classifier = pipeline("text-classification", model="./fine_tuned_sentiment", tokenizer=tokenizer)

# Dự đoán
predictions = classifier(df_test["combined_text"].tolist())

# Lấy nhãn
df_test["target"] = [label_id[pred["label"]] for pred in predictions]

# Lưu kết quả
df_test[["id", "target"]].to_csv("test_predictions.csv", index=False)

print("Dự đoán hoàn tất! Kết quả lưu vào test_predictions.csv")
